***Simulation Question 4:***

Use 80 percent of the CIFAR-10 training data to train your model.
This will serve as your baseline model

**Importing needed libraries**

In [ ]:
# Importing the libraries
from random import shuffle
import torch.optim as optim
import torch
from torchvision import datasets
import matplotlib.pyplot as plt
import torchvision.transforms as transforms
import os
from torch.utils.data import Dataset, DataLoader, random_split, TensorDataset
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression as LR
from datetime import datetime
from sklearn.preprocessing import StandardScaler

**Given base model**

In [ ]:
class CIFAR10Classifier(nn.Module):
    def __init__(self):
        super(CIFAR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(6272, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return x

**Define Load data to get the needed Data**

In [ ]:
def get_data():
    # CIFAR-10 dataset mean and standard deviation
    cifar10_mean = np.array([0.49421428, 0.48513139, 0.45040909])
    cifar10_std = np.array([0.24665252, 0.24289226, 0.26159238])

    # CIFAR-10 dataset transforms
    transform_train = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
    ])

    # CIFAR-10 dataset transforms
    transform_test = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(cifar10_mean, cifar10_std),
    ])

    # Unnormalize transform for CIFAR-10 dataset
    unnormalize_transform = transforms.Normalize(-cifar10_mean/cifar10_std, 1/cifar10_std)

    # CIFAR-10 dataset loading
    cifar10_dataset = datasets.CIFAR10(root='dataset', train=True, download=True, transform=transform_train)
    train_dataset, val_dataset = random_split(cifar10_dataset, [45000, 5000])
    test_dataset = datasets.CIFAR10(root='dataset', train=False, download=True, transform=transform_test)
    train_loader = DataLoader(dataset=train_dataset, batch_size=64, shuffle=True)
    val_loader = DataLoader(dataset=val_dataset, batch_size=64, shuffle=True)
    test_loader = DataLoader(dataset=test_dataset, batch_size=64, shuffle=True)

    return train_loader, val_loader, test_loader

train_loader, val_loader, test_loader = get_data()
print(len(train_loader))
print(len(val_loader))
print(len(test_loader))

100%|██████████| 170498071/170498071 [00:03<00:00, 44334842.36it/s]


Extracting dataset/cifar-10-python.tar.gz to dataset
Files already downloaded and verified
704
79
157


**define model_train function**

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def model_train(model, train_loader, val_loader, criterion, num_epochs, regularization_strength = None, model_name = None):
    optimizer = optim.Adam(model.parameters(), lr=0.001)
    if regularization_strength == None:
        regularization_strength = 0

    train_loss_arr, val_loss_arr = [], []
    train_acc_arr, val_acc_arr = [], []
    for epoch in range(num_epochs):
        train_loss, val_loss = .0, .0
        train_acc, val_acc = .0, .0

        model.train()
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            l2_reg = sum(torch.sum(param ** 2) for param in model.parameters())
            loss += regularization_strength * l2_reg
            train_loss += loss.item() * images.size(0)
            train_acc += torch.sum(torch.max(outputs, axis=1)[1] == labels).cpu().item()
            loss.backward()
            optimizer.step()

        model.eval()
        with torch.no_grad():
            for images, labels in val_loader:
                images, labels = images.to(device), labels.to(device)
                outputs = model(images)
                loss = criterion(outputs, labels)
                l2_reg = sum(torch.sum(param ** 2) for param in model.parameters())
                loss += regularization_strength * l2_reg
                val_loss += loss.item() * images.size(0)
                val_acc += torch.sum(torch.max(outputs, axis=1)[1] == labels).cpu().item()

        train_loss /= len(train_loader.dataset)
        val_loss /= len(val_loader.dataset)
        train_acc /= len(train_loader.dataset)
        val_acc /= len(val_loader.dataset)

        train_loss_arr.append(train_loss)
        val_loss_arr.append(val_loss)
        train_acc_arr.append(train_acc)
        val_acc_arr.append(val_acc)

        print(f"[Epoch {epoch}]\t"
            f"[{datetime.now().strftime('%H:%M:%S')}]\t"
            f"Train Loss: {train_loss:.4f}\t"
            f"Train Accuracy: {train_acc:.2f}\t"
            f"Validation Loss: {val_loss:.4f}\t\t"
            f"Validation Accuracy: {val_acc:.2f}")
        torch.save(model.state_dict(), f'{model_name}.pth')


**Train the base line model using the given data in this section**

In [ ]:
model = CIFAR10Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
num_epochs=10

model_train(model, train_loader, val_loader, criterion, num_epochs)

[Epoch 0]	[23:41:17]	Train Loss: 1.8088	Train Accuracy: 0.33	Validation Loss: 1.5114		Validation Accuracy: 0.45
[Epoch 1]	[23:41:40]	Train Loss: 1.5788	Train Accuracy: 0.42	Validation Loss: 1.3684		Validation Accuracy: 0.51
[Epoch 2]	[23:42:05]	Train Loss: 1.4768	Train Accuracy: 0.46	Validation Loss: 1.2742		Validation Accuracy: 0.55
[Epoch 3]	[23:42:28]	Train Loss: 1.4160	Train Accuracy: 0.48	Validation Loss: 1.2144		Validation Accuracy: 0.57
[Epoch 4]	[23:42:52]	Train Loss: 1.3678	Train Accuracy: 0.51	Validation Loss: 1.2056		Validation Accuracy: 0.58
[Epoch 5]	[23:43:15]	Train Loss: 1.3416	Train Accuracy: 0.52	Validation Loss: 1.1426		Validation Accuracy: 0.59
[Epoch 6]	[23:43:38]	Train Loss: 1.3214	Train Accuracy: 0.53	Validation Loss: 1.1378		Validation Accuracy: 0.60
[Epoch 7]	[23:44:02]	Train Loss: 1.3112	Train Accuracy: 0.53	Validation Loss: 1.1135		Validation Accuracy: 0.61
[Epoch 8]	[23:44:25]	Train Loss: 1.2937	Train Accuracy: 0.54	Validation Loss: 1.1192		Validation Accurac

**Check model accuracy on test_loader**

In [ ]:
# Evaluate on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 64.99%


**Training phase**

***Simulation Question 5:***

Train your baseline model with privacy enhancements. This is your
modified model. Ensure that the test accuracy difference between your baseline model and the
modified model is less than 15

**Methods used for privacy enhancements:**

    rounding output of model to 3 digits
    Restriction of prediction vector to top 3 elements
    Using regulatization term λ||Θ||

**These methods help the model leak less information when classifying**

In [ ]:
class Private_CIFAR10Classifier(nn.Module):
    def __init__(self):
        super(Private_CIFAR10Classifier, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)

        # Calculate the input size for the fully connected layer
        self._to_linear = None
        self._get_conv_output_size()

        self.fc1 = nn.Linear(self._to_linear, 64)
        self.fc2 = nn.Linear(64, 10)

    def _get_conv_output_size(self):
        x = torch.rand(1, 3, 32, 32)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)
        self._to_linear = x.numel()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        # only giving top k values
        k = 2
        topk_values, topk_indices = torch.topk(x, k, dim=1)
        mask = torch.zeros_like(x)
        mask.scatter_(1, topk_indices, topk_values)

        # Round the values in the mask to 3 decimal places
        rounded_mask = torch.round(mask * 100) / 100

        return mask


**Start training phase for the new model created as private model from the same data had from earlier**

In [ ]:
model = Private_CIFAR10Classifier().to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
regularization_strength = 5e-3  # Start with a small regularization strength
num_epochs = 10

model_train(model, train_loader, val_loader, criterion, num_epochs, regularization_strength)


[Epoch 0]	[09:09:00]	Train Loss: 2.2165	Train Accuracy: 0.26	Validation Loss: 2.1201		Validation Accuracy: 0.31
[Epoch 1]	[09:10:17]	Train Loss: 2.0763	Train Accuracy: 0.33	Validation Loss: 2.0028		Validation Accuracy: 0.37
[Epoch 2]	[09:15:12]	Train Loss: 2.0352	Train Accuracy: 0.35	Validation Loss: 1.9737		Validation Accuracy: 0.38


KeyboardInterrupt: 

**Check private model accuracy on the test_loader**

In [ ]:
# Evaluate on the test set
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

print(f"Test Accuracy: {100 * correct / total}%")

Test Accuracy: 17.4%


***Simulation Question 6.***

**Train two Attacker Models based on MIA techniques learned
in Phase 0, one for the baseline model and one for the modified model. Compare the MIA
accuracy of these two attacker models. Use 80 percent of the training data as your seen data,
and the remaining training data along with the test data as your unseen data**

**Create a shadow model to mimic the target model**

In [ ]:
class ShadowModel(nn.Module):
    def __init__(self):
        super(ShadowModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.fc1 = nn.Linear(6272, 64)
        self.fc2 = nn.Linear(64, 10)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 2)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        return F.softmax(x, dim=1)

**create private shadow model to mimic private model**

In [ ]:
class Private_ShadowModel(nn.Module):
    def __init__(self):
        super(Private_ShadowModel, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, 1)
        self.conv2 = nn.Conv2d(16, 32, 3, 1)
        self.dropout1 = nn.Dropout(0.25)
        self.dropout2 = nn.Dropout(0.5)
        self.pool = nn.MaxPool2d(2, 2)

        # Calculate the input size for the fully connected layer
        self._to_linear = None
        self._get_conv_output_size()

        self.fc1 = nn.Linear(self._to_linear, 64)
        self.fc2 = nn.Linear(64, 10)

    def _get_conv_output_size(self):
        x = torch.rand(1, 3, 32, 32)
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)
        self._to_linear = x.numel()

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = self.conv2(x)
        x = F.relu(x)
        x = self.pool(x)
        x = self.dropout1(x)
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc2(x)
        # only giving top k values
        k = 2
        topk_values, topk_indices = torch.topk(x, k, dim=1)
        mask = torch.zeros_like(x)
        mask.scatter_(1, topk_indices, topk_values)

        # Round the values in the mask to 3 decimal places
        rounded_mask = torch.round(mask * 100) / 100

        return mask




**Train shadow model in this section for base line model**

In [ ]:
# Train 10 different shadow models, each with its corresponding label dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
shadow_model = ShadowModel().to(device)
num_epochs = 5
criterion = nn.CrossEntropyLoss()


print(f"Training shadow model ...")
model_train(shadow_model, train_loader, val_loader, criterion, num_epochs)

# Save the shadow models
torch.save(shadow_model.state_dict(), f'shadow_model_Q6.pth')

# Example: Print summary of one shadow model
print(shadow_model)

**Create datas suitable for tranining the Attacker model these datas are created by using main datas on shadow models that mimic the target model**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to combine inputs and outputs into new dataset entries
def create_combined_data(shadow_models, loaders_by_label, mode):
    for label in range(10):
        shadow_model = shadow_models[label]
        shadow_model.eval()

        with torch.no_grad():
            for images, true_outputs in loaders_by_label[label]:
                images = images.to(device)
                outputs = shadow_model(images).cpu()
                for true_output, output in zip(true_outputs, outputs):
                    true_output, output = true_output.to(device), output.to(device)
                    # Ensure true_output and output have the same number of dimensions
                    if true_output.dim() == 0:  # Handle scalar tensor case
                        true_output = true_output.unsqueeze(0)
                    if output.dim() > 1:  # Handle higher-dimensional output tensor
                        output = output.squeeze()  # Adjust dimensions as needed
                    combined_output = torch.cat((true_output, output), dim=0)
                    yield (combined_output, mode)
                torch.cuda.empty_cache()

# Initialize the dictionary for the shadow models
shadow_models = {}

model_dir = './'

# Load shadow models
for i in range(10):
    model_path = os.path.join(model_dir, f'shadow_model_{i}.pth')
    shadow_model = ShadowModel().to(device)
    shadow_model.load_state_dict(torch.load(model_path, map_location=device))
    shadow_model.eval()  # Set the model to evaluation mode
    shadow_models[i] = shadow_model

# Example: Print the keys of the shadow_models dictionary to verify
print("Loaded shadow models:", shadow_models.keys())

# Create seen data
combined_in_data1 = list(create_combined_data(shadow_models, train_loaders_by_label, 1))
combined_in_data2 = list(create_combined_data(shadow_models, val_loaders_by_label, 1))
combined_in_data = combined_in_data1 + combined_in_data2

# Create unseen data
combined_out_data = list(create_combined_data(shadow_models, test_loaders_by_label, -1))



Loaded shadow models: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


***Simulation Question 7.***

**Improve your attacker models to achieve better MIA accuracy for
both the baseline and modified models (e.g., by increasing the number of shadow models).
Then compare the new accuracies with the previous results.**

**Split data based on their true outputs in order to train shadow models using the new seperated dataset**

In [ ]:
# Function to split dataset into parts based on labels
def split_dataset_by_labels(loader, num_labels=10):
    split_datasets = {i: [] for i in range(num_labels)}

    for images, labels in loader:
        images, labels = images.to(device), labels.to(device)
        for i in range(num_labels):
            mask = (labels == i)
            if mask.any():
                split_datasets[i].append((images[mask], labels[mask]))

    # Concatenate tensors within each label
    for label in split_datasets:
        if split_datasets[label]:
            images_list, labels_list = zip(*split_datasets[label])
            split_datasets[label] = TensorDataset(torch.cat(images_list), torch.cat(labels_list))
        else:
            split_datasets[label] = TensorDataset(torch.empty(0, 3, 32, 32), torch.empty(0, dtype=torch.long))

    return split_datasets

# Create data loaders for each label
train_loaders_by_label = {i: DataLoader(split_dataset_by_labels(train_loader)[i], batch_size=64, shuffle=False) for i in range(10)}
print('1')
val_loaders_by_label = {i: DataLoader(split_dataset_by_labels(val_loader)[i], batch_size=64, shuffle=False) for i in range(10)}
print('2')
test_loaders_by_label = {i: DataLoader(split_dataset_by_labels(test_loader)[i], batch_size=64, shuffle=False) for i in range(10)}


1
2


**Verify dataloaders have been split and see their shapes**

In [ ]:
# Example: Print the size of each label's dataset
for i in range(10):
    print(f"Label {i}: Train dataset size = {len(train_loaders_by_label[i])}, Val dataset size = {len(val_loaders_by_label[i])}, Test dataset size = {len(test_loaders_by_label[i])}")

Label 0: Train dataset size = 70, Val dataset size = 9, Test dataset size = 16
Label 1: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 2: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 3: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 4: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 5: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 6: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 7: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 8: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16
Label 9: Train dataset size = 71, Val dataset size = 8, Test dataset size = 16


**Train shadow models and save them**

In [ ]:
# Train 10 different shadow models, each with its corresponding label dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
shadow_models = {}
num_epochs = 5
criterion = nn.CrossEntropyLoss()

for label in range(10):
    shadow_model = ShadowModel().to(device)
    print(f"Training shadow model for label {label}...")
    model_train(shadow_model, train_loaders_by_label[label], val_loaders_by_label[label], criterion, num_epochs)
    shadow_models[label] = shadow_model

# Save the shadow models
for label in range(10):
    torch.save(shadow_models[label].state_dict(), f'shadow_model_{label}.pth')

# Example: Print summary of one shadow model
print(shadow_models[0])

Training shadow model for label 0...
[Epoch 0]	[23:52:02]	Train Loss: 1.4850	Train Accuracy: 0.99	Validation Loss: 1.4612		Validation Accuracy: 1.00
[Epoch 1]	[23:52:02]	Train Loss: 1.4612	Train Accuracy: 1.00	Validation Loss: 1.4612		Validation Accuracy: 1.00
[Epoch 2]	[23:52:02]	Train Loss: 1.4612	Train Accuracy: 1.00	Validation Loss: 1.4612		Validation Accuracy: 1.00
[Epoch 3]	[23:52:03]	Train Loss: 1.4612	Train Accuracy: 1.00	Validation Loss: 1.4612		Validation Accuracy: 1.00
[Epoch 4]	[23:52:03]	Train Loss: 1.4612	Train Accuracy: 1.00	Validation Loss: 1.4612		Validation Accuracy: 1.00
Training shadow model for label 1...
[Epoch 0]	[23:52:03]	Train Loss: 1.4845	Train Accuracy: 0.99	Validation Loss: 1.4612		Validation Accuracy: 1.00
[Epoch 1]	[23:52:04]	Train Loss: 1.4612	Train Accuracy: 1.00	Validation Loss: 1.4612		Validation Accuracy: 1.00
[Epoch 2]	[23:52:04]	Train Loss: 1.4612	Train Accuracy: 1.00	Validation Loss: 1.4612		Validation Accuracy: 1.00
[Epoch 3]	[23:52:05]	Train Los

**Create datas suitable for tranining the Attacker model these datas are created by using main datas on shadow models that mimic the target model**

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# Function to combine inputs and outputs into new dataset entries
def create_combined_data(shadow_models, loaders_by_label, mode):
    for label in range(10):
        shadow_model = shadow_models[label]
        shadow_model.eval()

        with torch.no_grad():
            for images, true_outputs in loaders_by_label[label]:
                images = images.to(device)
                outputs = shadow_model(images).cpu()
                for true_output, output in zip(true_outputs, outputs):
                    true_output, output = true_output.to(device), output.to(device)
                    # Ensure true_output and output have the same number of dimensions
                    if true_output.dim() == 0:  # Handle scalar tensor case
                        true_output = true_output.unsqueeze(0)
                    if output.dim() > 1:  # Handle higher-dimensional output tensor
                        output = output.squeeze()  # Adjust dimensions as needed
                    combined_output = torch.cat((true_output, output), dim=0)
                    yield (combined_output, mode)
                torch.cuda.empty_cache()

# Initialize the dictionary for the shadow models
shadow_models = {}

model_dir = './'

# Load shadow models
for i in range(10):
    model_path = os.path.join(model_dir, f'shadow_model_{i}.pth')
    shadow_model = ShadowModel().to(device)
    shadow_model.load_state_dict(torch.load(model_path, map_location=device))
    shadow_model.eval()  # Set the model to evaluation mode
    shadow_models[i] = shadow_model

# Example: Print the keys of the shadow_models dictionary to verify
print("Loaded shadow models:", shadow_models.keys())

# Create seen data
combined_in_data1 = list(create_combined_data(shadow_models, train_loaders_by_label, 1))
combined_in_data2 = list(create_combined_data(shadow_models, val_loaders_by_label, 1))
combined_in_data = combined_in_data1 + combined_in_data2

# Create unseen data
combined_out_data = list(create_combined_data(shadow_models, test_loaders_by_label, -1))



Loaded shadow models: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


**Print a few sample datas to see their looks**

In [ ]:
# Example: Print first few entries of each data
for i in range(5):
    input_data, label = combined_in_data[i]
    print(f"Input: {input_data}, Label: {label}")

for i in range(5):
    input_data, label = combined_out_data[i]
    print(f"Input: {input_data}, Label: {label}")

print(len(combined_in_data))
print(len(combined_out_data))

Input: tensor([0.0000e+00, 1.0000e+00, 6.4908e-36, 8.6705e-40, 6.4716e-37, 4.3246e-33,
        4.5849e-40, 7.0012e-38, 9.4511e-40, 2.2673e-36, 4.2039e-45],
       device='cuda:0'), Label: 1
Input: tensor([0.0000e+00, 1.0000e+00, 1.8384e-39, 1.1210e-43, 1.4268e-40, 1.5921e-36,
        1.0930e-43, 7.7408e-42, 2.1720e-43, 6.0685e-40, 0.0000e+00],
       device='cuda:0'), Label: 1
Input: tensor([0.0000e+00, 1.0000e+00, 4.5651e-25, 9.2733e-28, 8.9754e-26, 3.6450e-23,
        7.6139e-28, 1.6806e-26, 1.1567e-27, 2.3961e-25, 2.0441e-31],
       device='cuda:0'), Label: 1
Input: tensor([0.0000e+00, 1.0000e+00, 4.0455e-30, 2.2786e-33, 6.2865e-31, 7.2405e-28,
        2.4053e-33, 6.8057e-32, 3.6128e-33, 1.6926e-30, 1.0643e-37],
       device='cuda:0'), Label: 1
Input: tensor([0.0000e+00, 1.0000e+00, 4.9468e-25, 9.0966e-28, 1.0268e-25, 4.1889e-23,
        9.4592e-28, 1.9552e-26, 1.2167e-27, 2.5655e-25, 2.3409e-31],
       device='cuda:0'), Label: 1
Input: tensor([0.0000e+00, 1.0000e+00, 1.8301e-34,

**create final dataset for the attacker model**

In [ ]:
# Step 1: Combine the seen and unseen data
def get_shadow_datasets(combined_in_data, combined_out_data):
    # print(len(combined_in_data))
    # print(len(combined_out_data))
    combined_dataset = combined_in_data + combined_out_data
    shuffle(combined_dataset)
    # print(len(combined_dataset))

    # Step 2: Split 60,000 data into 50,000 (train/validation) and 10,000 (test)
    train_val_data = combined_dataset[:50000]
    test_data = combined_dataset[50000:]

    # Step 3: Further split train_val_data into 45,000 (train) and 5,000 (validation)
    train_data = train_val_data[:45000]
    val_data = train_val_data[45000:]

    # Create DataLoaders
    batch_size = 64

    train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader, test_loader

train_shadow_loader, val_shadow_loader, test_shadow_loader = get_shadow_datasets(combined_in_data, combined_out_data)

# Example: Print sizes to verify
# print(f"Total Combined Dataset Size: {len(combined_dataset)}")
print(f"Training Data Size: {len(train_shadow_loader)}")
print(f"Validation Data Size: {len(val_shadow_loader)}")
print(f"Test Data Size: {len(test_shadow_loader)}")

Training Data Size: 704
Validation Data Size: 79
Test Data Size: 157


**see a sample data**

In [ ]:
print(f'sample train data: {train_shadow_loader.dataset[0]}')

sample train data: (tensor([6.0000e+00, 0.0000e+00, 2.9567e-42, 0.0000e+00, 0.0000e+00, 2.9937e-41,
        0.0000e+00, 1.0000e+00, 0.0000e+00, 0.0000e+00, 0.0000e+00],
       device='cuda:0'), -1)


**preprocess data to train linear regression model as attacker model**

In [ ]:
# Function to extract data and labels from DataLoader
def extract_data_and_labels(loader):
    data = []
    labels = []
    for x,y in loader:
        data.append(x.cpu().numpy())  # Assuming DataLoader returns PyTorch tensors
        labels.append(y.cpu().numpy())
    data = np.concatenate(data, axis=0)
    labels = np.concatenate(labels, axis=0)
    # normalizing
    scaler = StandardScaler()
    scaler.fit(data)
    scaled_data = scaler.transform(data)
    return scaled_data, labels

X_train, y_train = extract_data_and_labels(train_shadow_loader)
X_val, y_val = extract_data_and_labels(val_shadow_loader)
X_test, y_test = extract_data_and_labels(test_shadow_loader)

**simple linear regression model as attacker**

In [ ]:
lr = LR()
lr.fit(X_train, y_train)
# Evaluate the model
y_pred = lr.predict(X_test)

**check attacker model accuracy**

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 83.63%


**create private shadow models to mimic private model**

In [ ]:
# Train 10 different shadow models, each with its corresponding label dataset
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
private_shadow_models = {}
regularization_strength = 5e-3
num_epochs = 5
criterion = nn.CrossEntropyLoss()

for label in range(10):
    private_shadow_model = Private_ShadowModel().to(device)
    print(f"Training shadow model for label {label}...")
    model_train(private_shadow_model, train_loaders_by_label[label], val_loaders_by_label[label], criterion, num_epochs, regularization_strength)
    private_shadow_models[label] = private_shadow_model

# Save the shadow models
for label in range(10):
    torch.save(private_shadow_models[label].state_dict(), f'private_shadow_model_{label}.pth')

# Example: Print summary of one shadow model
print(private_shadow_models[0])

Training shadow model for label 0...
[Epoch 0]	[00:13:35]	Train Loss: 0.9489	Train Accuracy: 0.98	Validation Loss: 0.3545		Validation Accuracy: 1.00
[Epoch 1]	[00:13:35]	Train Loss: 0.2338	Train Accuracy: 1.00	Validation Loss: 0.1461		Validation Accuracy: 1.00
[Epoch 2]	[00:13:36]	Train Loss: 0.1174	Train Accuracy: 1.00	Validation Loss: 0.0913		Validation Accuracy: 1.00
[Epoch 3]	[00:13:36]	Train Loss: 0.0854	Train Accuracy: 1.00	Validation Loss: 0.0755		Validation Accuracy: 1.00
[Epoch 4]	[00:13:36]	Train Loss: 0.0753	Train Accuracy: 1.00	Validation Loss: 0.0701		Validation Accuracy: 1.00
Training shadow model for label 1...
[Epoch 0]	[00:13:37]	Train Loss: 0.8505	Train Accuracy: 0.99	Validation Loss: 0.2999		Validation Accuracy: 1.00
[Epoch 1]	[00:13:37]	Train Loss: 0.1928	Train Accuracy: 1.00	Validation Loss: 0.1180		Validation Accuracy: 1.00
[Epoch 2]	[00:13:37]	Train Loss: 0.0964	Train Accuracy: 1.00	Validation Loss: 0.0754		Validation Accuracy: 1.00
[Epoch 3]	[00:13:38]	Train Los

**read saved private shadow models and prepare data for attacker model**

In [ ]:
# Initialize the dictionary for the shadow models
private_shadow_models = {}

model_dir = './'

# Load shadow models
for i in range(10):
    model_path = os.path.join(model_dir, f'private_shadow_model_{i}.pth')
    private_shadow_model = Private_ShadowModel().to(device)
    private_shadow_model.load_state_dict(torch.load(model_path, map_location=device))
    private_shadow_model.eval()  # Set the model to evaluation mode
    private_shadow_models[i] = private_shadow_model

# Example: Print the keys of the shadow_models dictionary to verify
print("Loaded private shadow models:", private_shadow_models.keys())

# Create seen data
private_combined_in_data1 = list(create_combined_data(private_shadow_models, train_loaders_by_label, 1))
private_combined_in_data2 = list(create_combined_data(private_shadow_models, val_loaders_by_label, 1))
private_combined_in_data = private_combined_in_data1 + private_combined_in_data2

# Create unseen data
private_combined_out_data = list(create_combined_data(private_shadow_models, test_loaders_by_label, -1))



Loaded private shadow models: dict_keys([0, 1, 2, 3, 4, 5, 6, 7, 8, 9])


**see samples of final data**

In [ ]:
# Example: Print first few entries of each data
for i in range(5):
    input_data, label = private_combined_in_data[i]
    print(f"Input: {input_data}, Label: {label}")

for i in range(5):
    input_data, label = private_combined_out_data[i]
    print(f"Input: {input_data}, Label: {label}")

print(len(private_combined_in_data))
print(len(private_combined_out_data))

Input: tensor([ 0.0000,  8.8220, -0.0367,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0'), Label: 1
Input: tensor([ 0.0000,  9.0015, -0.0374,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0'), Label: 1
Input: tensor([ 0.0000,  7.0305, -0.0292,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0'), Label: 1
Input: tensor([ 0.0000,  7.8352, -0.0327,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0'), Label: 1
Input: tensor([ 0.0000,  7.1656, -0.0298,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0'), Label: 1
Input: tensor([ 0.0000,  8.0605, -0.0335,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0'), Label: -1
Input: tensor([ 0.0000,  8.8565, -0.0368,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0

**combine in and out datas**

In [ ]:
private_train_shadow_loader, private_val_shadow_loader, private_test_shadow_loader = get_shadow_datasets(private_combined_in_data, private_combined_out_data)

# Example: Print sizes to verify
# print(f"Total Combined Dataset Size: {len(combined_dataset)}")
print(f"Training Data Size: {len(private_train_shadow_loader)}")
print(f"Validation Data Size: {len(private_val_shadow_loader)}")
print(f"Test Data Size: {len(private_test_shadow_loader)}")

Training Data Size: 704
Validation Data Size: 79
Test Data Size: 157


**private attacker model sample train data**

In [ ]:
print(f'sample train data: {private_train_shadow_loader.dataset[0]}')

sample train data: (tensor([ 0.0000,  8.7030, -0.0361,  0.0000,  0.0000,  0.0000,  0.0000,  0.0000,
         0.0000,  0.0000,  0.0000], device='cuda:0'), 1)


**Seperate parts of data for linear regression model**

In [ ]:
# Extract data and labels from DataLoader
private_X_train, private_y_train = extract_data_and_labels(private_train_shadow_loader)
private_X_val, private_y_val = extract_data_and_labels(private_val_shadow_loader)
private_X_test, private_y_test = extract_data_and_labels(private_test_shadow_loader)

**Train linear regression attacker model for private dataset**

In [ ]:
lr = LR()
lr.fit(private_X_train, private_y_train)
# Evaluate the model
private_y_pred = lr.predict(private_X_test)

**check accuracy of private attacker model**

In [ ]:
accuracy = accuracy_score(private_y_test, private_y_pred)
print("Accuracy: {:.2f}%".format(accuracy * 100))

Accuracy: 82.71%


In [ ]:
# class AttackerModel(nn.Module):
#     def __init__(self):
#         super(AttackerModel, self).__init__()
#         self.fc1 = nn.Linear(11, 128)
#         self.fc2 = nn.Linear(128, 64)
#         self.fc3 = nn.Linear(64, 32)
#         self.fc4 = nn.Linear(32, 1)
#         self.sigmoid = nn.Sigmoid()

#     def forward(self, x):
#         x = torch.relu(self.fc1(x))
#         x = torch.relu(self.fc2(x))
#         x = torch.relu(self.fc3(x))
#         x = self.sigmoid(self.fc4(x))
#         return x
